<center>
NNTI Assignment 1

November 2, 2023

Name: Aleksey Morshnev
Student ID: 7042691
Email: almo00008@stud.uni-saarland.de
</center>

In [2]:
# List of packages you want to check/install
packages_to_install = ["numpy", "pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia", "pandas"]

In [4]:
%%time
import importlib

for package_name in packages_to_install:
    try:
        importlib.import_module(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        print(f"{package_name} is not installed. Installing it now...")
        !conda install -y {package_name}

numpy is already installed.
pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia is not installed. Installing it now...
Solving environment: done

# All requested packages already installed.

pandas is already installed.
CPU times: user 336 ms, sys: 76.5 ms, total: 412 ms
Wall time: 8.68 s


# Exercise 1

In [5]:
import numpy as np

In [8]:
x = np.random.rand(4, 16)
x

array([[0.01190693, 0.87022059, 0.58015836, 0.07428869, 0.22856679,
        0.48783667, 0.59254467, 0.30421009, 0.55053576, 0.20669143,
        0.66265152, 0.08431609, 0.92115346, 0.73728237, 0.69132713,
        0.95439298],
       [0.4861322 , 0.47334901, 0.24201772, 0.02567615, 0.84054477,
        0.30557098, 0.10188062, 0.44165726, 0.65692405, 0.01226265,
        0.89205496, 0.14240437, 0.64027398, 0.74317733, 0.61420711,
        0.03485847],
       [0.44627544, 0.52254899, 0.89545339, 0.16705646, 0.93070682,
        0.53403745, 0.23128577, 0.13052196, 0.40448578, 0.45592348,
        0.88116362, 0.44840204, 0.74034402, 0.12612517, 0.75985806,
        0.56870976],
       [0.42026578, 0.98942966, 0.71258575, 0.6469206 , 0.70970922,
        0.56675464, 0.37964427, 0.65095473, 0.41198335, 0.13462611,
        0.9831249 , 0.55368896, 0.08495565, 0.63569914, 0.99262257,
        0.77780829]])

In [10]:
w1, w2, w3 = [np.random.rand(*shape) for shape in [(16, 16), (32, 16), (2, 32)]]
w1, w2, w3

(array([[0.17812665, 0.31807452, 0.0936939 , 0.31687584, 0.01287833,
         0.21434645, 0.54328993, 0.62963907, 0.90002795, 0.07833271,
         0.73711588, 0.07052915, 0.75355975, 0.96916294, 0.32599305,
         0.680202  ],
        [0.36420106, 0.34481347, 0.48473504, 0.16199199, 0.03226853,
         0.94281601, 0.613635  , 0.0843384 , 0.02496037, 0.57180832,
         0.13692371, 0.8695926 , 0.64625588, 0.36638773, 0.22241589,
         0.70389316],
        [0.26080982, 0.90018536, 0.34262873, 0.63478502, 0.7853562 ,
         0.68229566, 0.94488109, 0.51980282, 0.23158599, 0.21763436,
         0.61513566, 0.48938606, 0.17896903, 0.73069386, 0.38183505,
         0.29858466],
        [0.20269472, 0.08208689, 0.39356951, 0.58086571, 0.6589575 ,
         0.97015013, 0.92767191, 0.79119823, 0.16203601, 0.37507124,
         0.04321167, 0.28167405, 0.07591373, 0.94784298, 0.49402502,
         0.38977749],
        [0.19076878, 0.00169588, 0.36656451, 0.28684805, 0.38460234,
         0.1536

In [12]:
result = ((x @ w1.T) @ w2.T) @ w3.T # Result of forward pass
result

array([[479.43407915, 376.29935969],
       [397.63409209, 313.20316999],
       [477.71504278, 372.698453  ],
       [570.11914193, 447.95394218]])

# (Bonus)

In [14]:
fourByFour = np.random.rand(4, 4)

In [16]:
eigenvalues, eigenvectors = np.linalg.eig(fourByFour)
eigenvalues, eigenvectors

(array([ 2.08066711+0.j        , -0.24884598+0.31897604j,
        -0.24884598-0.31897604j,  0.23872874+0.j        ]),
 array([[ 0.38447128+0.j        ,  0.2006644 -0.36526j   ,
          0.2006644 +0.36526j   ,  0.21268957+0.j        ],
        [ 0.52007861+0.j        ,  0.7085714 +0.j        ,
          0.7085714 -0.j        , -0.58162667+0.j        ],
        [ 0.54545421+0.j        , -0.48010555+0.10258107j,
         -0.48010555-0.10258107j, -0.42209664+0.j        ],
        [ 0.53308515+0.j        ,  0.07867237+0.27754628j,
          0.07867237-0.27754628j,  0.66204833+0.j        ]]))

# Exercise 2

## Part1

In [17]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [20]:
class CustomDataset(Dataset):
    def __init__(self, json_file, train=True, transform=None):
        with open(json_file, 'r') as f:
            data = json.load(f)

        # Split data into training and testing datasets
        split_ratio = 0.8
        split_index = int(len(data) * split_ratio)
        if train:
            self.data = data[:split_index]
        else:
            self.data = data[split_index:]

        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample

In [22]:
def custom_collate_fn(batch):
    max_seq_len = max(len(sample['tokens']) for sample in batch)

    padded_inputs = []
    labels = []
    for sample in batch:
        input_sequence = sample['tokens']
        label = sample['label']

        padding = [0] * (max_seq_len - len(input_sequence))
        padded_input_sequence = input_sequence + padding

        padded_inputs.append(padded_input_sequence)
        labels.append(label)

    inputs = torch.tensor(padded_inputs)
    labels = torch.tensor(labels)

    return inputs, labels


In [24]:
class CustomDataLoader(DataLoader):
    def __init__(self, dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None):
        super(CustomDataLoader, self).__init__(
            dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn
        )

    def __iter__(self):
        for batch in super(CustomDataLoader, self).__iter__():
            yield batch

In [26]:
dataset_path = './data/Dataset1/data.json'

train_dataset = CustomDataset(dataset_path, train=True)
train_loader = CustomDataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

for batch in train_loader:
    inputs, outputs = batch
    
test_dataset = CustomDataset(dataset_path, train=False)
test_loader = CustomDataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

for batch in test_loader:
    inputs, outputs = batch

## Part 2

In [28]:
import pandas as pd

In [30]:
class SickDataset(Dataset):
    def __init__(self, csv_file, train=True):
        self.data = pd.read_csv(csv_file)
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = torch.tensor(self.data.iloc[idx, :-1], dtype=torch.float32)

        target = torch.tensor(pd.get_dummies(self.data.iloc[idx, -1]).values, dtype=torch.float32)

        return inputs, target

In [32]:
class SickDataLoader(DataLoader):
    def __init__(self, dataset, batch_size=1, shuffle=False, num_workers=0):
        super(SickDataLoader, self).__init__(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    def __iter__(self):
        for batch in super(SickDataLoader, self).__iter__():
            inputs, outputs = zip(*batch)
            yield torch.stack(inputs), torch.stack(outputs)

In [33]:
sick_dataset_path = './data/Dataset2/data.csv'

sick_dataset = SickDataset(sick_dataset_path, train=True)

custom_dataloader = CustomDataLoader(sick_dataset, batch_size=32, shuffle=True)

for batch in custom_dataloader:
    inputs, outputs = batch

# Ignore warnings

/tmp/ipykernel_1582/1200310341.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inputs = torch.tensor(self.data.iloc[idx, :-1], dtype=torch.float32)
/tmp/ipykernel_1582/1200310341.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inputs = torch.tensor(self.data.iloc[idx, :-1], dtype=torch.float32)
/tmp/ipykernel_1582/1200310341.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inputs = torch.tensor(self.data.iloc[idx, :-1]